In [4]:
import numpy as np

class Muon:
    def __init__(self, theta, phi, z, v):
        self.theta = theta
        self.phi = phi
        self.z_list = [z]
        self.v = v
        self.x_list = [10]
        self.y_list = [10]
        self.lifetime = 3

    def propagate(self, time_step):
        time = 0
        self.x_list.append(self.v * time_step * np.sin(self.phi) * np.cos(self.theta))
        self.y_list.append(self.v * time_step * np.sin(self.phi) * np.sin(self.theta))
        self.z_list.append(self.v * time_step * np.cos(self.phi))

# Defines 3D regions where scintillation occurs.
class Scintillator:
    def __init__(self, r, z):
        self.r = r
        self.z = z

    def has_muon(self, muon):
        #TODO: make work with traced path for a muon.
        return self.z[0] <= muon.z <= self.z[1] and np.sqrt(muon.x**2 + muon.y**2) < self.r

experiment = [Scintillator(1, [10,9]), Scintillator(1, [8,0])]

def simulate_path(muon, scintillators, time_step):
    time = 0
    hits = []
    #TODO: Figure this out.
    while time < muon.lifetime:
        muon.propagate(time_step)
        for scintillator in scintillators:
            if scintillator.has_muon(muon):
                hits =


In [1]:
def monte_carlo(N):
    return

SyntaxError: invalid syntax (Temp/ipykernel_7208/2442406903.py, line 9)